Copyright (c). All rights reserved.

Licensed under the MIT License.

# Troubleshooting HPO for fine-tuning pre-trained language models

## 1. Introduction

In this notebook, we demonstrate a procedure for troubleshooting HPO failure in fine-tuning pre-trained language models (introduced in the following paper):

*[An Empirical Study on Hyperparameter Optimization for Fine-Tuning Pre-trained Language Models](https://arxiv.org/abs/2106.09204). Xueqing Liu, Chi Wang. ACL-IJCNLP 2021*

Notes:

*In this notebook, we only run each experiment 1 time for simplicity, which is different from the paper (3 times). To reproduce the paper's result, please run 3 repetitions and take the average scores.

*Running this notebook takes about one hour.

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the `notebook` and `nlp` options:

```bash
pip install flaml[nlp]==0.7.1 # in higher version of flaml, the API for nlp tasks changed
```

Our paper was developed under transformers version 3.4.0. We uninstall and reinstall transformers==3.4.0:


In [11]:
%pip install flaml[nlp]==0.7.1 # in higher version of flaml, the API for nlp tasks changed
%pip install transformers==3.4.0
from flaml.nlp import AutoTransformers


## 2. Initial Experimental Study


### Load dataset 

Load the dataset using AutoTransformer.prepare_data. In this notebook, we use the Microsoft Research Paraphrasing Corpus (MRPC) dataset and the Electra model as an example:

In [2]:
autohf = AutoTransformers()
preparedata_setting = {
                          "dataset_subdataset_name": "glue:mrpc",
                          "pretrained_model_size": "google/electra-base-discriminator:base",
                          "data_root_path": "data/",
                          "max_seq_length": 128,
                       }
autohf.prepare_data(**preparedata_setting)


console_args has no attribute pretrained_model_size, continue
console_args has no attribute dataset_subdataset_name, continue
console_args has no attribute algo_mode, continue
console_args has no attribute space_mode, continue
console_args has no attribute search_alg_args_mode, continue
console_args has no attribute algo_name, continue
console_args has no attribute pruner, continue
console_args has no attribute resplit_mode, continue
console_args has no attribute rep_id, continue
console_args has no attribute seed_data, continue
console_args has no attribute seed_transformers, continue
console_args has no attribute learning_rate, continue
console_args has no attribute weight_decay, continue


Reusing dataset glue (/home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-6a78e5c95406457c.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-e8d0f3e04c3b4588.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-4b0966b394994163.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-6a78e5c95406457c.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c996572

### Running grid search

First, we run grid search using Electra. By specifying `algo_mode="grid"`, AutoTransformers will run the grid search algorithm. By specifying `space_mode="grid"`, AutoTransformers will use the default grid search configuration recommended by the Electra paper:

In [3]:
import transformers
autohf_settings = {
                      "resources_per_trial": {"gpu": 1, "cpu": 1},
                      "num_samples": 1,
                      "time_budget": 100000,  # unlimited time budget
                      "fp16": True,
                      "algo_mode": "grid",  # set the search algorithm to grid search
                      "space_mode": "grid", # set the search space to the recommended grid space
                      "transformers_verbose": transformers.logging.ERROR
                   }
validation_metric, analysis = autohf.fit(**autohf_settings)

2021-06-16 10:45:35,071	INFO tune.py:450 -- Total run time: 106.56 seconds (106.41 seconds for the tuning loop).


Total running time: 106.57789206504822 seconds


Get the time for running grid search: 

In [4]:
GST = autohf.last_run_duration
print("grid search for {} took {} seconds".format(autohf.jobid_config.get_jobid_full_data_name(), GST))

grid search for glue_mrpc took 106.57789206504822 seconds


After the HPO run finishes, generate the predictions and save it as a .zip file to be submitted to the glue website. Here we will need the library AzureUtils which is for storing the output information (e.g., analysis log, .zip file) locally and uploading the output to an azure blob container (e.g., if multiple jobs are executed in a cluster). If the azure key and container information is not specified, the output information will only be saved locally. 

In [5]:
predictions, test_metric = autohf.predict()
from flaml.nlp import AzureUtils

print(autohf.jobid_config)

azure_utils = AzureUtils(root_log_path="logs_test/", autohf=autohf)
azure_utils.write_autohf_output(valid_metric=validation_metric,
                                predictions=predictions,
                                duration=GST)
print(validation_metric)

remove_columns_ is deprecated and will be removed in the next major version of datasets. Use the dataset.remove_columns method instead.


Cleaning the existing label column from test data


JobID(dat=['glue'], subdat='mrpc', mod='grid', spa='grid', arg='dft', alg='grid', pru='None', pre_full='google/electra-base-discriminator', pre='electra', presz='base', spt='ori', rep=0, sddt=43, sdhf=42, var1=None, var2=None)
Your output will not be synced to azure because azure key and container name are not specified
The path for saving the prediction .zip file is not specified, setting to data/ by default
Your output will not be synced to azure because azure key and container name are not specified
{'eval_accuracy': 0.7254901960784313, 'eval_f1': 0.8276923076923076, 'eval_loss': 0.516851007938385}


The validation F1/accuracy we got was 92.4/89.5. After the above steps, you will find a .zip file for the predictions under data/result/. Submit the .zip file to the glue website. The test F1/accuracy we got was 90.4/86.7. As an example, we only run the experiment one time, but in general, we should run the experiment multiple repetitions and report the averaged validation and test accuracy.

### Running Random Search

Next, we run random search with the same time budget as grid search:

In [6]:
def tune_hpo(time_budget, this_hpo_space):
    autohf_settings = {
                          "resources_per_trial": {"gpu": 1, "cpu": 1},
                          "num_samples": -1,
                          "time_budget": time_budget,
                          "fp16": True,
                          "algo_mode": "hpo",  # set the search algorithm mode to hpo
                          "algo_name": "rs",
                          "space_mode": "cus", # customized search space (this_hpo_space)
                          "hpo_space": this_hpo_space,
                          "transformers_verbose": transformers.logging.ERROR
                       }
    validation_metric, analysis = autohf.fit(**autohf_settings)
    predictions, test_metric = autohf.predict()
    azure_utils = AzureUtils(root_log_path="logs_test/", autohf=autohf)
    azure_utils.write_autohf_output(valid_metric=validation_metric,
                                    predictions=predictions,
                                    duration=GST)
    print(validation_metric)

In [7]:
hpo_space_full = {
                      "learning_rate": {"l": 3e-5, "u": 1.5e-4, "space": "log"},
                      "warmup_ratio": {"l": 0, "u": 0.2, "space": "linear"},
                      "num_train_epochs": [3],
                      "per_device_train_batch_size": [16, 32, 64],
                      "weight_decay": {"l": 0.0, "u": 0.3, "space": "linear"},
                      "attention_probs_dropout_prob": {"l": 0, "u": 0.2, "space": "linear"},
                      "hidden_dropout_prob": {"l": 0, "u": 0.2, "space": "linear"},
                  }

tune_hpo(GST, hpo_space_full)

(pid=50964) {'eval_loss': 0.5942569971084595, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.10434782608695652}
(pid=50964) {'eval_loss': 0.5942569971084595, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.10434782608695652}
(pid=50948) {'eval_loss': 0.649192214012146, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.2}
(pid=50948) {'eval_loss': 0.649192214012146, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.2}


2021-06-16 10:48:21,624	INFO tune.py:450 -- Total run time: 114.32 seconds (109.41 seconds for the tuning loop).


Total running time: 114.35665488243103 seconds


Your output will not be synced to azure because azure key and container name are not specified
The path for saving the prediction .zip file is not specified, setting to data/ by default
Your output will not be synced to azure because azure key and container name are not specified
{'eval_accuracy': 0.7328431372549019, 'eval_f1': 0.8320493066255777, 'eval_loss': 0.5411379933357239}


The validation F1/accuracy we got was 93.5/90.9. Similarly, we can submit the .zip file to the glue website. The test F1/accuaracy we got was 81.6/70.2. 

## 3. Troubleshooting HPO Failures

Since the validation accuracy is larger than grid search while the test accuracy is smaller, HPO has overfitting. We reduce the search space:

In [8]:
hpo_space_fixwr = {
                       "learning_rate": {"l": 3e-5, "u": 1.5e-4, "space": "log"},
                       "warmup_ratio": [0.1],
                       "num_train_epochs": [3],
                       "per_device_train_batch_size": [16, 32, 64],
                       "weight_decay": {"l": 0.0, "u": 0.3, "space": "linear"},
                       "attention_probs_dropout_prob": {"l": 0, "u": 0.2, "space": "linear"},
                       "hidden_dropout_prob": {"l": 0, "u": 0.2, "space": "linear"},
                   }
tune_hpo(GST, hpo_space_fixwr)

(pid=54411) {'eval_loss': 0.624100387096405, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5}
(pid=54411) {'eval_loss': 0.624100387096405, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5}
(pid=54411) {'eval_loss': 0.624100387096405, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5}
(pid=54417) {'eval_loss': 0.5938675999641418, 'eval_accuracy': 0.7156862745098039, 'eval_f1': 0.8258258258258258, 'epoch': 0.5}
(pid=54417) {'eval_loss': 0.5938675999641418, 'eval_accuracy': 0.7156862745098039, 'eval_f1': 0.8258258258258258, 'epoch': 0.5}
(pid=54417) {'eval_loss': 0.5938675999641418, 'eval_accuracy': 0.7156862745098039, 'eval_f1': 0.8258258258258258, 'epoch': 0.5}


2021-06-16 10:51:34,598	INFO tune.py:450 -- Total run time: 151.57 seconds (136.77 seconds for the tuning loop).


Total running time: 151.59901237487793 seconds


Your output will not be synced to azure because azure key and container name are not specified
The path for saving the prediction .zip file is not specified, setting to data/ by default
Your output will not be synced to azure because azure key and container name are not specified
{'eval_accuracy': 0.7475490196078431, 'eval_f1': 0.8325203252032519, 'eval_loss': 0.5056071877479553}


The validation F1/accuracy we got was 92.6/89.7, the test F1/accuracy was 85.9/78.7, therefore overfitting still exists and we further reduce the space: 

In [9]:
hpo_space_min = {
                   "learning_rate": {"l": 3e-5, "u": 1.5e-4, "space": "log"},
                   "warmup_ratio": [0.1],
                   "num_train_epochs": [3],
                   "per_device_train_batch_size": [16, 32, 64],
                   "weight_decay": [0.0],
                   "attention_probs_dropout_prob": [0.1],
                   "hidden_dropout_prob": [0.1],
                }
tune_hpo(GST, hpo_space_min)

(pid=57835) {'eval_loss': 0.5822290778160095, 'eval_accuracy': 0.7058823529411765, 'eval_f1': 0.8181818181818181, 'epoch': 0.5043478260869565}
(pid=57835) {'eval_loss': 0.5822290778160095, 'eval_accuracy': 0.7058823529411765, 'eval_f1': 0.8181818181818181, 'epoch': 0.5043478260869565}
(pid=57835) {'eval_loss': 0.5822290778160095, 'eval_accuracy': 0.7058823529411765, 'eval_f1': 0.8181818181818181, 'epoch': 0.5043478260869565}
(pid=57835) {'eval_loss': 0.5822290778160095, 'eval_accuracy': 0.7058823529411765, 'eval_f1': 0.8181818181818181, 'epoch': 0.5043478260869565}
(pid=57836) {'eval_loss': 0.6087244749069214, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.10344827586206896}
(pid=57836) {'eval_loss': 0.6087244749069214, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.10344827586206896}
(pid=57836) {'eval_loss': 0.6087244749069214, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.103448275862068

2021-06-16 10:54:14,542	INFO tune.py:450 -- Total run time: 117.99 seconds (112.99 seconds for the tuning loop).


Total running time: 118.01927375793457 seconds


Your output will not be synced to azure because azure key and container name are not specified
The path for saving the prediction .zip file is not specified, setting to data/ by default
Your output will not be synced to azure because azure key and container name are not specified
{'eval_accuracy': 0.7181372549019608, 'eval_f1': 0.8174962292609351, 'eval_loss': 0.5494586229324341}


The validation F1/accuracy we got was 90.4/86.7, test F1/accuracy was 83.0/73.0. Since the validation accuracy is below grid search, we increase the budget to 4 * GST:

In [10]:
hpo_space_min = {
                   "learning_rate": {"l": 3e-5, "u": 1.5e-4, "space": "log"},
                   "warmup_ratio": [0.1],
                   "num_train_epochs": [3],
                   "per_device_train_batch_size": [32],
                   "weight_decay": [0.0],
                   "attention_probs_dropout_prob": [0.1],
                   "hidden_dropout_prob": [0.1],
                 }
tune_hpo(4 * GST, hpo_space_min)

(pid=61251) {'eval_loss': 0.6236899495124817, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5}
(pid=61251) {'eval_loss': 0.6236899495124817, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5}
(pid=61251) {'eval_loss': 0.6236899495124817, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5}
(pid=61251) {'eval_loss': 0.6236899495124817, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5}
(pid=61251) {'eval_loss': 0.6236899495124817, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5}
(pid=61255) {'eval_loss': 0.6249027848243713, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.3}
(pid=61255) {'eval_loss': 0.6249027848243713, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.3}
(pid=61255) {'eval_loss': 0.6249027848243713, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.81222707

2021-06-16 11:03:23,308	INFO tune.py:450 -- Total run time: 507.09 seconds (445.79 seconds for the tuning loop).


Total running time: 507.15925645828247 seconds


Your output will not be synced to azure because azure key and container name are not specified
The path for saving the prediction .zip file is not specified, setting to data/ by default
Your output will not be synced to azure because azure key and container name are not specified
{'eval_accuracy': 0.7401960784313726, 'eval_f1': 0.8333333333333334, 'eval_loss': 0.5303606986999512}


The validation F1/accuracy we got was 93.5/91.1, where the accuracy outperforms grid search. The test F1/accuracy was 90.1/86.1. As a result, random search with 4*GST and the minimum space overfits. We stop the troubleshooting process because the search space cannot be further reduced.